In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import gym
import gym_disaster
import numpy as np

## Sub-Environment Definition

In its simplest form, we need the sub-environment, `Disaster-v0`, to have the following characteristics:

- The Sub-Environment should be ran by a Sub-Agent (e.g., A2C)
- The env should produce, as observations, information only pertinent to itself (e.g., it's unaware of the other sub-environment instances)
- The action the environment accepts should come from the allocation agent

In itself, the environment should be self-sustaining as an environment suitable for RL.  It's recieved actions should only be *adjusted* by the allocation agent, not determined.  The associated sub-agent determines the actual action.

For our Snow Plow example, the finite resource we need to allocate is available plows.  Each sub-environment represents a region that may require plowing.  Reward is dictated by the accumulation of snow and the number of plows present.

In [32]:
env = gym.make('Disaster-v0')
env.reset()
env.render()

Iteration: 0
Current Plows: 0.0
Current Accumulation: 0.0
Realized Fall: 0.0
Forecasted Mean: 2
Forecasted Variance: 0.704732434179131
Reward: None


In [33]:
def run(env, policy, verbose=False):
    obs = env.reset()
    verbose and print(obs)
    plows = policy(obs)
    verbose and env.render()
    verbose and print()
    done = False
    total_reward = 0.

    while not done:
        obs, reward, done, info = env.step(plows)
        plows = policy(obs)
        verbose and print(obs)
        verbose and env.render()
        verbose and print()

        total_reward += reward
        if done:
            verbose and print('Total Reward:', total_reward)
            break
    return total_reward

In [35]:
results = {}

def test(func):
    env = gym.make('Disaster-v0')
    global results
    results[func.__name__] = {
        'data': [],
        'func': func
    }
    for i in range(100):
        results[func.__name__]['data'].append(run(env, results[func.__name__]['func']))

def random_policy(obs):
    return np.random.choice(env.env.total_plows)
    
def trivial_policy(num):
    def tp(obs):
        return num
    tp.__name__ = f'triv_policy_{num}'
    return tp

def simple_policy(obs):
    plows = obs[0] * env.env.total_plows
    accum = obs[1] * env.env.max_accum
    mean = obs[2] * env.env.max_fall
    var = obs[3] * env.env.max_forecast_var

    base = 0
    if accum > 10:
        return base + 3
    elif accum > 5:
        return base + 2
    elif accum > 0:
        return base + 1
    else:
        if mean < 2:
            return base
        else:
            return base + 1

test(random_policy)
for num in range(env.env.total_plows+1):
    test(trivial_policy(num))
test(simple_policy)

for key in results:
    print(key, '\t', np.mean(results[key]['data']))

random_policy 	 -483.5089631372183
triv_policy_0 	 -3416.019265603014
triv_policy_1 	 -1078.6756352792497
triv_policy_2 	 -448.19018729572
triv_policy_3 	 -336.90086011052864
triv_policy_4 	 -393.85779280250836
triv_policy_5 	 -472.3504074135186
triv_policy_6 	 -571.294985683546
simple_policy 	 -648.9536114880871


## Resource Allocation Considerations

Switching gears, we now have to consider the allocation agent's task and how it relates to this sub-agent.  To begin, we consider the simple example where the sub-environment is a simple stock trading environment where the observation space is data pertinent to forecasting the future asset value (relative to it's current value), the action space is a discrete space of two actions (long and neutral), and the reward is a surrogate for the return on investment per time step.

At any time step, the sub-agent will observe the data and decide to either be long or neutral in the asset.  If the sub-agent goes long, then the sub-agent will recieve a positive reward if the value of the asset increases during the next time step, the reward will be negative if the value decreases, and the reward will be zero if the asset's value doesn't change.

To relate this to the real-world, this task is similar to given someone some amount of cash (say, $100), and having them look at the stock's chart every hour and deciding to either invest all of their available cash in a specific stock XYZ or to hold on to their cash (i.e., if the person thinks the price of XYZ is going to increase in the next hour, they will buy $100 worth of XYZ and reasses their position in an hour).

Now, the allocation agent is tasked with allocating resources to sub-agents, and is agnostic to the actual sub-environment and sub-agent it is creating allocations for.  This makes the allocation agent a manager of the sub-agents, forecasting how well a sub-agent will perform (given pertinent observation data) relative to other sub-agents, and allocating resources to maximize it's own reward.

In the running example, we could imagine a number of different sub-agent's trading different assets (or possibly even the same asset), with all operating in the same way working to maximize their own reward.  At any time step, one sub-agent might be in a position where they have the potential to gain a greater reward than the other, or one is more likely to be profitable, etc.  Each individual sub-agent is unaware of this: they are not concerned with the operations of the other sub-agents and only attempt to maximize their own reward.  This is where the allocation agent comes in.

The allocation agent observes the operations of all the sub-agents (including how well they are performing, how they perform relative to one another, how they perform given specific observations, etc).  The allocation agent then allocates a percentage of the available assets to each sub-agent, based on it's prediction of how well they will do during the time step.  The allocation agent will allocate more resources to agent's it thinks will result in the highest reward, which may change (possibly considerably) after every time step.

In our example, the sub-agents may be acting as a team of independent stock traders while the allocation agent is their manager.  The allocation agent has a finite amount of resources (e.g., available cash), and must decide how much cash to give each agent at every point in time in order to maximize profits.  If the manager feels one agent is in a position to perform relatively better than another, the manager will choose to allocate more cash to that agent.  The manager, however, will want to diversify in order to reduce risk (and increase average, accumulated rewards), so the manager will probably not allocate all of it's resoruces to only one agent, and instead will distribute the resources among the agents in a way that would give more resources to agents it forecasts to do better.

To simplify matters, the sub-agents operate under the assumption that the control one unit of a continuous resource.  In our example, the sub-agent doesn't actually control any captial; it merely makes suggestions as to whether or not going long would be more profitable than being neutral (i.e., either 0% invested or 100% invested).  The allocation agent is the one that actually determines how much of it's resources are to be allocated to the sub-agent (e.g., if the manager controls \$1000 and thinks a specific sub-agent will be profitable, they may give the sub-agent 50% of it's captial to trade, or $500).

## Plowing as an Investment

With all this being said, we need to construct our environment to run on similar logic.  That is, each sub-agent needs to be operating in it's own sub-environment, independent and agnostic to one another.  The sub-agent attempts to maximize some reward which can be interpretted as a return on investment during some period of time.  The allocation agent must then determine which sub-agents are more likely to produce the most reward, and allocate more of it's resources to them.  In our case, the finite resource that the allocation agent controls is the number of plows, and each sub-agent is attempting to utilize plows to reduce snow accumulation in a specific region.

The analogy to trading assets can be maintained by defining a proper return on investment, which is less natural in this case.  For trading assets (such as stocks), return is easily calculated as the final price of the asset, minus trading costs, divided by the original price (so if at $t_0$ stock XYZ was trading at $100 and an agent goes long, then, if at $t_1$ XYZ is trading at $110 and there is no cost to trade, the return will be 10%).  For plowing snow, we need to formulate these pieces to produce a similar value.

An example of how this could work: let's say we have a city split into 4 regions (A, B, C, D), and our goal is to minimize the snow accumulation throughout the city.  We have four plows available, each of which can occupy one region at a time (with the possibility of having multiple plows in a single region).  The plows reduce snow accumulation in the region they are in for the time steps they are present.  Then the naive approach would be to give each region a single plow and call it a day.  This solution, however, may not be optimal.

Let's assume that a plow can reduce the average snow accumulation in a region by 1" per time step.  So, if two plows are in a single region during a time step, they will reduce the average accumulation in that region by 2".  Now, if we go with the naive approach, then each region will have their snow accumulation reduced by 1" every time step.  If we value the minmization of snow accumulation in each region equally, and each region recieves the same amount of snow fall per time step, then this solution would be valid (and optimal, albeit trivially). But these assumptions might not hold.

Let's say that, at a given time step, region A and B are to recieve 2" of snow while regions C and D are to recieve 0".  Then our naive solution would be suboptimal (with the optimal solution being allocating two plows to A and B each).  So we see that the allocation agent's task is non-trivial given more realistic data.  On top of this, the value we place on the minimization of snow accumulation may be different per region and may possibly change per time step: if region A is downtown and region B is a rural area, and if the time is 7:00 AM on a weekday, then reducing the accumulation in region A may be seen as more valuable compared to B given the greater amount of traffic region A would be recieving at that time.

Another consideration is the constraints we need to put on the sub-agents in each region.  Trivially, each sub-agent will attempt to allocate 100% of the plows to their region, if they are not experiencing any accumulation as this would be the most "profitable" approach regardless of the observation.  This, of course, would not work in real life.  In reality, we have a finite number of plows that need to be properly distributed between regions, so we certainly couldn't make all the regions happy all the time.  To counter this, we can associate a cost with allocating plows to their region, with the idea being that a sub-agent will have to balance the cost of the plow and the "profit" of the snow removal so as to "take what they only need."

This balancing is, in itself, not a trivial task and will require a fair amount of analysis.  But, assuming we can properly formulate this cost, we can have the sub-agents allocating an optimal number of plows for their region.

## Agent

We see we can make simple policies to make decisions about how many plows to allocate to the region, but with the environment potentially being complex, this task becomes time consuming and computationally expensive to perform.  We instead train an Actor-Critic agent to learn to maximize the reward.

In [36]:
from curls.framework import SessionManager, ActorCriticAgent, get_discounted_rewards

In [37]:
agent = ActorCriticAgent(env, batch_size=64)
session = SessionManager()

In [38]:
for sess in range(10):
    batch = {
        "observations": [],
        "actions": [],
        "discounted_rewards": []
    }

    episode_iteration = 1
    episode_iterations = 100

    total_rewards = []
    while episode_iteration <= episode_iterations or len(batch['observations']) > 0:
        observations, actions, rewards, dones, infos = session.run(agent, env)

        batch['observations'] += observations
        batch['actions'] += actions
        batch['discounted_rewards'] += get_discounted_rewards(rewards, agent.gamma)
        episode_iteration += 1

        if len(batch['observations']) >= agent.batch_size:
            for epoch in range(agent.epochs):
                agent.learn(**batch)
            batch = {
                "observations": [],
                "actions": [],
                "discounted_rewards": []
            }
            total_rewards.append(np.sum(rewards))

    print(sess, np.mean(total_rewards))

0 -502.5765446158387
1 -378.70139825609573
2 -531.7063309895989
3 -503.9768079730031
4 -436.25307849886156
5 -407.5726192294591
6 -361.0140298146468
7 -639.784779290138
8 -395.92667476232833
9 -295.27322800317876


In [47]:
batch = {
    "observations": [],
    "actions": [],
    "discounted_rewards": []
}

episode_iteration = 1
episode_iterations = 100

total_rewards = []
while episode_iteration <= episode_iterations or len(batch['observations']) > 0:
    observations, actions, rewards, dones, infos = session.run(agent, env)

    batch['observations'] += observations
    batch['actions'] += actions
    batch['discounted_rewards'] += get_discounted_rewards(rewards, agent.gamma)
    episode_iteration += 1

    if len(batch['observations']) >= agent.batch_size:
        batch = {
            "observations": [],
            "actions": [],
            "discounted_rewards": []
        }
        total_rewards.append(np.sum(rewards))

print(np.mean(total_rewards))

-550.8192623164783


# Rethinking Markov Decision Processes

In a Markov Decision Process, we have an agent and an environment who interact through actions, observations, and rewards.  The process typically works as follows:

    1) The environment outputs the current state's observation
    2) The agent receives the observation for the current state, and responds by returning an action
    3) The environment receives the action, and returns the observation for the next state as well as the reward for the action taken during the previous state

This process involves the agent taking one of some number of actions at any given state.  The action that's chosen at a given state is determined by the agent's policy.  For the sake of simplicity, we'll assume the agent is an Actor-Critic agent, so that it's policy as a probability distrubution over the possible actions.  The agent recieves the observation, and the probabilities for each action (ideally) correspond to the advantage of each action.

For example, if at some state the agent's policy returns `[0.3, 0.5, 0.2]`, then the action corresponding to `0.5` is the action that should lead to the highest advantage.  If we sample from this distrubution, then we expect the agent to select the action with the highest probability the most frequently (this solves the exploration/exploitation issue to some degree).

Now, during training, the agent is ran through a simulation in which it performs actions and receives rewards so as to find how those actions performed.  If the agent takes an action that results in a positive advantage, the agent is encouraged to take the action more often.  In practice, this occurs by calculating the advantage for taking that action at the state (possible since we'd have simulated the action reward results at this point) and using it to adjust the probabilities using cross-entropy loss in a neural network.

In terms of training, the "predicted" value is equal to the masked action vector multiplied by the advantage.  For example, if at some state the action probability is `[0.3, 0.5, 0.2]` and the agent chooses the second action (whose probability is `0.5`) and this results in an advantage of `+1`, then the "predicted" value used for training is `[0, 1, 0]`.  This will "push" the probability of the second action higher while pushing the other two lower.  On the other hand, if, in a similar situation, the advantage is `-1`, then the "predicted" value used for training would be `[0, -1, 0]`, pushing the value lower (note: these values are negative logged, so the actual loss is `-log([0, 1, 0])` in the first case).

This works well, but is not optimal for every process.  For example, by "masking" the other two possible actions, we are telling the agent that their advantages are 0.  This isn't necessarily true.  Consider an example where the advantage for the second and the third actions are the same (say, `+1`).  Then we are throwing away this information and training the agent on what is essentially incorrect data.  Of course, during training, we expect the agent to sample from these actions, allowing it find that these two advantages are similar and adjusting it's values accordingly.

With that being said, if we can calculate the advantage for every action, then we can (trivially) train the agent to always select the best one.  The calculation of the advantage, however, is dependent on the estimation of the value by the critic.  So, if the critic can't accurately compute the value, then we can't accurately compute the advantage.

On top of this, the process of simulating the environment for every path is computationally expensive.  If we tried to simulate every sequence of actions we could take, it would take `(# of actions)^(# of steps)` step simulations to perform.  If we were to just simulate the next step, this would instead take `(# of actions) * (# of steps)`.

So, we can improve the accuracy of the critic by making it's task easier.  The future discounted reward is calculated as $V_t = r_t + \gamma * V_{t+1}$.  We set $\gamma$ to correspond to the uncertainty of future rewards, which, in typical applications, is set to 0.99.  This, to me, stands out as problematic.  With $\gamma$ being 0.99, we giving a fair amount of weight to states that are so far out we'd have no chance of accurately predicting it in realistic environments.  The value of 0.99 has worked well in more deterministic environments, such as Atari games or physics simulations, but it has been noted the the values predicted by the critic tend to be substantially inaccurate (despite the process working relatively well).

I propose a "smarter" approach to calculating values.  If we scale the discounting factor to correspond to how accurate we can make future predictions, we can reduce the number of states the critic can accurately account for drastically, allowing us to simulate the environment in the branching approach proposed before.

For example, let's say the value is directly dependent on our ability to forecast hourly accumulation.  Let's say that the forecast produces as Gaussian distubution, with a typical variance of 1".  So, if we forecast the accumulation to be 3" in the next hour, we're likely to see accumulation in the range of 2" to 4".  This means that if we compute the value based on these forecasts, we can very rapidly find ourselves estimating inaccurate values, e.g., at only two steps out, the accumulation can either be from 1" to 5", at 3 steps it's 0" to 6", and so on (assuming high enough variance).

In this case, the critic is going to do a poor job prediciting the value of the current state if we try to take into account too many steps out.  If, instead, we limit the value to only consider a few steps (either by making $\gamma$ small or possibly a more complex function of the state $\Gamma$), then we can not only increase the accuracy of the critic, but we can also simulate steps in the environment for every given action without the time complexity blowing up.

For example, if we have 5 actions and we are confident in the critic predicting the value over the next 5 steps, then we can run the simulation at every state for every action with $5^5 = 3125$ step simulations.  This is not computationally easy, but it is feasible, and although this would be costly to run over the number of episodes typically required to train an RL agent, we will see vast improvements in the quality of training per episode, possibly offsetting the added cost to some degree.

## From the Bottom Up

In order to work through the changes required to train a successful allocation agent, we have to consider the differences in a typical Markov Decision Process and the new Allocation Process.

First, consider a normal MDP:

<img src="http://billy-inn.github.io/images/rl1.1.png"></img>

In practice, the state at timestep t $S_t$ is an observation, or a tensor of some predetermined shape which includes information about the given state which the Agent is expected to make inferences from.



http://billy-inn.github.io/images/rl1.1.png


















